In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as py
import time
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import create_engine

In [11]:
covid_file = "CSV/covid_us_county.csv"
population_file="CSV/US_population.csv"

covid_df = pd.read_csv(covid_file)

population_df=pd.read_csv(population_file)

covid_df.head()

,fips,county,state,lat,long,date,cases,state_code,deaths
0,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-22,0,AL,0
1,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-23,0,AL,0
2,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-24,0,AL,0
3,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-25,0,AL,0
4,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-26,0,AL,0


In [12]:
population_df.head()

,state,Population
0,United States,"328,239,523"
1,Alabama,"4,903,185"
2,Alaska,"731,545"
3,Arizona,"7,278,717"
4,Arkansas,"3,017,804"


In [13]:
#Calculating the avrage rate of infection for each state
case = covid_df.groupby(["state"]).max()["cases"]
mean_state=covid_df.groupby(["state"]).mean()["cases"]

covid_by_state=pd.DataFrame({"Cases":case,
                             "Mean" :mean_state}).reset_index()
covid_by_state




,state,Cases,Mean
0,Alabama,83039,3587.029276
1,Alaska,32413,874.319037
2,American Samoa,0,0.000000
3,Arizona,573082,23094.614915
4,Arkansas,44983,2001.059092
5,California,1276254,27537.130389
6,Colorado,75412,3331.617100
7,Connecticut,101119,14530.079781
8,Delaware,65530,8946.099818
9,Diamond Princess,49,44.080146


In [15]:
# merged twp data set

merge_df = pd.merge(covid_by_state , population_df, on="state")

merge_df["Infection Rate"]=(merge_df["Cases"]/merge_df["Population"]*100)
merge_df=merge_df.sort_values('Infection Rate', ascending=False)
merge_df                        
#merge_df.to_csv("../CSV/Infection_rate.csv" )

TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [16]:
#calculating the top ten states in term of infection rate
Ten_top=merge_df.nlargest(10,"Infection Rate")
Ten_top

KeyError: 'Infection Rate'

In [ ]:
#grouping  by month 

covid_df.index = pd.to_datetime(covid_df['date'])
                                #format='%y-%m-%d %I:%M%p')
covid_df.groupby(by=[covid_df.index.month, covid_df.index.year])
covid_df.head()

In [ ]:
#seprating date value from yyyy-mm-dd to seprate columns
covid_df['day'] = covid_df.index.day
covid_df['month'] = covid_df.index.month
covid_df['year'] = covid_df.index.year
covid_df

In [ ]:
#monitoring the rate of infection in 4 randomly chosen states during 2020
States_df=covid_df.loc[(covid_df["state"]=="Florida")|(covid_df["state"]=="Indiana")|(covid_df["state"]=="Wisconsin")|(covid_df["state"]=="Louisiana")]
cases=States_df.groupby(["year","month","state"]).max()["cases"]

unimpacted_states=pd.DataFrame({
                            "Infection Rate":cases,
                            }).reset_index()
unimpacted_states.drop(unimpacted_states[unimpacted_states['year'] == 2021].index, inplace = True)
unimpacted_states["month"].replace({1: "January",2: "February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}, inplace=True)

unimpacted_states



### Create database connection

In [ ]:
connection_string = "postgres:postgres@localhost:5432/Infection_Rate"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

### Load DataFrames into database

In [ ]:
merge_df.to_sql(name='statepopulation', con=engine, if_exists='append', index=True)